# Intro to Deep Learning

## Perceptron

In [6]:
import torch
import torch.nn as nn

class Perceptron(nn.Module):
    def __init__(self, threshold=5):
        super().__init__()
        self.weight = nn.Parameter(torch.randn(1))
        self.bias = nn.Parameter(torch.randn(1))
        #self.heaviside_step_function = lambda x : x < threshold
        self.threshold = threshold
        
    def forward(self, x):
        x = self.weight * x
        x += self.bias
        #x = x < self.threshold
        x = x.float()
        #x = self.heaviside_step_function(x).float()
        return x
    
net = Perceptron()

In [7]:
from torch.utils.data import DataLoader
from torch.utils.data import TensorDataset

x = torch.randint(-10, 10, size=(1000,))
y = x > 5
tr = TensorDataset(x, y)
tloader = DataLoader(tr, batch_size=4, shuffle=True)

x = torch.randint(0, 10, size=(1000,))
y = x > 5
va = TensorDataset(x, y)
vloader = DataLoader(va, batch_size=4, shuffle=True)

In [8]:
# We will use a simple Mean Squared Error
def mse(preds, target):
  se = ((preds - target.float()) ** 2).sum()
  mse = se / len(preds)
  print (mse)
  return nn.Parameter(mse)

# We will use a simple Mean Squared Error
def mse(preds, target):
    preds, target = preds.float(), target.float()
    loss = torch.numel(preds) - (preds == target).sum()
    return nn.Parameter(loss.float())

In [9]:
import torch.optim as optim

def train(net, crit, trl, val, epochs=5, lr=0.01, clf=True, bclf=-1, dev='cuda'):
  # Initialize the Optimizer
  opt = optim.Adam(net.parameters(), lr=lr)
  dev = 'cuda' if torch.cuda.is_available() and dev == 'cuda' else 'cpu'
  net.to(dev)
  for i in range(epochs):
    for data, labl in trl:
      data, labl = data.to(dev), labl.to(dev)
      if clf and bclf != -1: labl = (labl == bclf).unsqueeze(1).float()
      out = net(data.float())
      #print (out, labl)
      loss = crit(out, labl.float())
      #print (loss)
      opt.zero_grad()
      loss.backward()
      opt.step()
    
    with torch.no_grad():
      acc = 0; vloss = 0;
      for data, labl in val:
        data, labl = data.to(dev), labl.to(dev)
        if clf and bclf != -1: labl = (labl == bclf).unsqueeze(1).float()
        out = net(data.float())
        vloss += crit(out, labl).detach().cpu().numpy().item()
        if clf: plabl = torch.argmax(out, dim=0)
        if clf and bclf != -1: plabl = plabl.reshape(-1, 1).float()
        if clf: acc += (labl.float() == plabl.float()).detach().cpu().sum().numpy()
      
      vloss /= (len(val) * val.batch_size)
      if clf:
        acc /= float(len(val))
        print ('Epoch {}/{} - Validation Loss {:.2f} - Accuracy - {:.2f}'.format(i+1, epochs,
          vloss, acc))
      else:
        print ('Epoch {}/{} - Validation Loss {:.2f}'.format(i+1, epochs, vloss))

In [10]:
train(net, mse, tloader, vloader, epochs=100, lr=0.01, dev='cpu', bclf=1)

Epoch 1/100 - Validation Loss 1.00 - Accuracy - 0.86
Epoch 2/100 - Validation Loss 1.00 - Accuracy - 0.81
Epoch 3/100 - Validation Loss 1.00 - Accuracy - 0.96
Epoch 4/100 - Validation Loss 1.00 - Accuracy - 0.90
Epoch 5/100 - Validation Loss 1.00 - Accuracy - 0.83
Epoch 6/100 - Validation Loss 1.00 - Accuracy - 0.94
Epoch 7/100 - Validation Loss 1.00 - Accuracy - 0.79
Epoch 8/100 - Validation Loss 1.00 - Accuracy - 0.84
Epoch 9/100 - Validation Loss 1.00 - Accuracy - 0.93
Epoch 10/100 - Validation Loss 1.00 - Accuracy - 0.64
Epoch 11/100 - Validation Loss 1.00 - Accuracy - 0.99
Epoch 12/100 - Validation Loss 1.00 - Accuracy - 0.94
Epoch 13/100 - Validation Loss 1.00 - Accuracy - 0.84
Epoch 14/100 - Validation Loss 1.00 - Accuracy - 0.75
Epoch 15/100 - Validation Loss 1.00 - Accuracy - 0.94
Epoch 16/100 - Validation Loss 1.00 - Accuracy - 0.81
Epoch 17/100 - Validation Loss 1.00 - Accuracy - 0.92
Epoch 18/100 - Validation Loss 1.00 - Accuracy - 0.88
Epoch 19/100 - Validation Loss 1.00 -